In [2]:
import itertools
import os

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

from experience import load_experience
from bclassification.utils_base import (
    print_class_weights,
    compute_weight_bias,
    print_dataset,
    plot_metrics,
    plot_cm,
    plot_roc,
    describe_results,
)
from lib.constants import Constants as Const
from lib.data_utils import (
    make_dir,
    env_pf,
    create_results_dir,
    save_dict_to_file,
)
from lib.tf_utils import (
    print_variables,
    ResidulaFCBlock,
    MatthewsCorrelationCoefficient,
)
from lib.visualizer import Visualizer, pprint

Visualizer()

experience_dir = make_dir(os.path.join(Const.EXPERIENCE_DIR, "data-aug"))
results_dir = make_dir(os.path.join(Const.RESULTS_DIR, "il"))

agent_name = "agent-mip"
case_name = "l2rpn_2019_art"
env_dc = True
verbose = False

case_results_dir = make_dir(os.path.join(results_dir, f"{case_name}-{env_pf(env_dc)}"))

In [3]:
"""
    Parameters
"""

random_seed = 1

input_mode = "structured"
label_mode = "dn"

n_window_targets = 12  # 0 or 12
n_window_history = 1
downsampling_rate = 0.10
n_window_forecasts = 1
use_actions = True
feature_scaling = True
batch_normalization = False

n_perturb = 0
scale_perturb = 0.02

val_frac = 0.10
test_frac = 0.10

# Model
model_type = "res"  # "fc" or "res"
dropout_rate = 0.1
l1_reg = 0
l2_reg = 0
n_hidden = 512
n_hidden_layers = 2
threshold = 0.50
pos_scaling = 1

# Training
learning_rate = 1e-5
n_batch = 512
n_epochs = 300

In [4]:
def perturb(X, Y, scale, n):
    n_gen = 5
    n_load = 11
    n_line = 20
    n_sub = 14
    
    x_p = []
    y_p = []
    for _ in range(n):
        if input_mode == "binary":
            # Observation size
            s = 3 * n_gen + n_load + 2 * n_line + (4 * n_line + 2 * n_gen + 2 * n_load) + n_sub + 2 * n_line

            # Features to perturb
            n_first = 3 * n_gen + n_load + 2 * n_line  # Flows and injections

            x = X.copy()
            for h in range(n_window_history + 1):
                noise = np.random.normal(loc=0.0, scale=scale, size=(x.shape[0], n_first))
                x[:, h * s : (h * s + n_first)] = x[:, h * s : (h * s + n_first)] + noise

            # Forecast
            if n_window_forecasts > 0:
                n_last = n_window_forecasts * (n_gen + n_load)
                noise = np.random.normal(loc=0.0, scale=scale, size=(x.shape[0], n_last))
                x[:, -n_last:] = x[:, -n_last:] + noise

        elif input_mode == "structured":
            # Observation size
            s = 2 * (n_gen + n_load) + 2 * n_gen + 5 * n_line + n_sub + 2 * n_line

            # Features to perturb
            n_first = 2 * (n_gen + n_load) + 2 * n_gen + 5 * n_line  # Flows and injections

            x = X.copy()
            for h in range(n_window_history + 1):
                noise = np.random.normal(loc=0.0, scale=scale, size=(x.shape[0], n_first))
                x[:, h * s : (h * s + n_first)] = x[:, h * s : (h * s + n_first)] + noise

            # Forecasts
            if n_window_forecasts > 0:
                n_last = n_window_forecasts * (n_gen + n_load)
                noise = np.random.normal(loc=0.0, scale=scale, size=(x.shape[0], n_last))
                x[:, -n_last:] = x[:, -n_last:] + noise
        
        x_p.append(x)
        y_p.append(Y)
    
    return np.vstack(x_p), np.hstack(y_p)

In [5]:
%%capture cap --no-stderr
"""
    Dataset
"""

np.random.seed(random_seed)
tf.random.set_seed(random_seed)

tar_str = f"w{n_window_targets}"
dr_str = str(int(100 * downsampling_rate))
f_str = str(n_window_forecasts)
h_str = str(n_window_history)

file_name = f"fc-d{dr_str}-h{h_str}-f{f_str}-{tar_str}-{input_mode}.npz"
file_path = os.path.join(experience_dir, file_name)

data = np.load(file_path)
X_all = data["X_all"]
Y_all = data["Y_all"]
mask_targets = data["mask_targets"]
X = X_all[mask_targets, :]
Y = Y_all[mask_targets]

class_weight, initial_bias = compute_weight_bias(Y)
initial_bias = 0
class_weight[1] = class_weight[1] * pos_scaling 

X_train, X_val, Y_train, Y_val = train_test_split(
    X, Y, test_size=val_frac, random_state=random_seed
)

X_train, X_test, Y_train, Y_test = train_test_split(
    X_train, Y_train, test_size=test_frac, random_state=random_seed
)

# Augment by adding random noise and upsampling the positive class
if scale_perturb > 0 and n_perturb > 0:
    indices_pos = np.equal(Y_train, 1)
    X_train_perturb, Y_train_perturb = perturb(X_train[indices_pos, :], Y_train[indices_pos], scale=scale_perturb, n=n_perturb)
    X_train = np.vstack((X_train, X_train_perturb))    
    Y_train = np.hstack((Y_train, Y_train_perturb))
    
mask_test_neg = np.logical_and(~mask_targets, np.random.binomial(1, val_frac, mask_targets.size).astype(np.bool))
X_test_all = np.concatenate((X_test, X_all[mask_test_neg, :]))
Y_test_all = np.concatenate((Y_test, Y_all[mask_test_neg]))

print_dataset(X_all, Y_all, "All data")
print_dataset(X, Y, "Data")
print_dataset(X_train, Y_train, "Train")
print_dataset(X_val, Y_val, "Validation")
print_dataset(X_test, Y_test, "Test")
print_dataset(X_test_all, Y_test_all, "Test-All")
print_class_weights(class_weight)
pprint("Initial bias:", "{:.4f}".format(float(initial_bias)))

model_dir = create_results_dir(case_results_dir, model_name=model_type)

del data
del X_all
del Y_all

In [6]:
"""
    Model
"""

metrics = [
    tf.keras.metrics.TruePositives(thresholds=threshold, name="tp"),
    tf.keras.metrics.FalsePositives(thresholds=threshold, name="fp"),
    tf.keras.metrics.TrueNegatives(thresholds=threshold, name="tn"),
    tf.keras.metrics.FalseNegatives(thresholds=threshold, name="fn"),
    tf.keras.metrics.BinaryAccuracy(threshold=threshold, name="accuracy"),
    tf.keras.metrics.Precision(thresholds=threshold, name="precision"),
    tf.keras.metrics.Recall(thresholds=threshold, name="recall"),
    MatthewsCorrelationCoefficient(threshold=threshold, name="mcc"),
    tf.keras.metrics.AUC(name="auc"),
]

if l1_reg > 0:
    kwargs_reg = {
        "kernel_regularizer": tf.keras.regularizers.L1(l1_reg),
        "bias_regularizer": tf.keras.regularizers.L1(l1_reg),
    }
elif l2_reg > 0:
    kwargs_reg = {
        "kernel_regularizer": tf.keras.regularizers.L2(l2=l2_reg),
        "bias_regularizer": tf.keras.regularizers.L2(l2=l2_reg),
    }
else:
    kwargs_reg = {}

input_dim = X.shape[-1]

tf.random.set_seed(random_seed)
if model_type == "fc":
    hidden_layers = [
        (
            tf.keras.layers.Dense(n_hidden, activation="relu", **kwargs_reg),
            tf.keras.layers.Dropout(dropout_rate),
        )
        for _ in range(n_hidden_layers)
    ]
    hidden_layers = list(itertools.chain(*hidden_layers))

    if batch_normalization:
        hidden_layers = hidden_layers + [tf.keras.layers.BatchNormalization()]
    
    model = tf.keras.Sequential(
        [
            tf.keras.layers.Dense(
                n_hidden, activation="relu", input_shape=(input_dim,), **kwargs_reg
            ),
            tf.keras.layers.Dropout(dropout_rate),
            *hidden_layers,
            tf.keras.layers.Dense(
                1,
                activation="sigmoid",
                bias_initializer=tf.keras.initializers.Constant(initial_bias),
                **kwargs_reg,
            ),
        ]
    )

elif model_type == "linear":
    model = tf.keras.Sequential(
        [
            tf.keras.layers.Dense(
                1,
                activation="sigmoid",
                bias_initializer=tf.keras.initializers.Constant(initial_bias), input_shape=(input_dim,),
                **kwargs_reg,
            ),
        ]
    )
else:
    hidden_layers = [
        (
            ResidulaFCBlock(n_hidden, activation="relu", **kwargs_reg),
            tf.keras.layers.Dropout(dropout_rate),
        )
        for _ in range(n_hidden_layers // 2)
    ]
    
    hidden_layers = list(itertools.chain(*hidden_layers))

    if batch_normalization:
        hidden_layers = hidden_layers + [tf.keras.layers.BatchNormalization()]
    
    model = tf.keras.Sequential(
        [
            tf.keras.layers.Dense(
                n_hidden, activation="relu", input_shape=(input_dim,), **kwargs_reg
            ),
            tf.keras.layers.Dropout(dropout_rate),
            *hidden_layers,
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.Dense(
                1,
                activation="sigmoid",
                bias_initializer=tf.keras.initializers.Constant(initial_bias),
                **kwargs_reg,
            ),
        ]
    )

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=metrics,
)

checkpoint_path = os.path.join(model_dir, "ckpts")
ckpt = tf.train.Checkpoint(model=model, optimizer=model.optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

pprint("Model directory:", model_dir)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    pprint("Restoring checkpoint from:", ckpt_manager.latest_checkpoint)
    
    
save_dict_to_file(
    {
        "random_seed": random_seed,
        "input_mode": input_mode,
        "label_mode": label_mode,
        "val_frac": val_frac,
        "downsampling_rate": downsampling_rate,
        "n_window_targets": n_window_targets,
        "n_window_history": n_window_history,
        "use_actions": use_actions,
        "feature_scaling": feature_scaling,
        "n_perturb": n_perturb,
        "scale_perturb": scale_perturb,
        "model_type": model_type,
        "dropout_rate": dropout_rate,
        "l1_reg": l1_reg,
        "l2_reg": l2_reg,
        "n_hidden": n_hidden,
        "n_hidden_layers": n_hidden_layers,
        "learning_rate": learning_rate,
        "n_batch": n_batch,
        "n_epochs": n_epochs,
        "threshold": threshold,
        "pos_scaling": pos_scaling,
        "batch_normalization": batch_normalization,
    },
    os.path.join(model_dir, "params.txt"),
)

Model directory:                        ./results\il\l2rpn_2019_art-dc\2020-11-04_15-54-52_res


In [7]:
cap.show()

with open(os.path.join(model_dir, "log.txt"), "a") as f:
    f.write(cap.stdout)

    - All data:                         X, Y	       (740873, 412), (740873,)
        - Positive labels:              1.03 %
        - Negative labels:              98.97 %

    - Data:                             X, Y	        (72947, 412), (72947,)
        - Positive labels:              10.45 %
        - Negative labels:              89.55 %

    - Train:                            X, Y	        (59086, 412), (59086,)
        - Positive labels:              10.33 %
        - Negative labels:              89.67 %

    - Validation:                       X, Y	         (7295, 412), (7295,)
        - Positive labels:              11.60 %
        - Negative labels:              88.40 %

    - Test:                             X, Y	         (6566, 412), (6566,)
        - Positive labels:              10.31 %
        - Negative labels:              89.69 %

    - Test-All:                         X, Y	        (73063, 412), (73063,)
        - Positive labels:              0.93 %
        - Nega

In [ ]:
"""
    Training
"""
# tensorboard_path = os.path.join(model_dir, "logs")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(
#     log_dir=tensorboard_path, write_graph=False, write_images=False, update_freq="epoch"
# )

# print(f"tensorboard --logdir={tensorboard_path}")

training = model.fit(
    X_train,
    Y_train,
    epochs=n_epochs,
    batch_size=n_batch,
    class_weight=class_weight,
    validation_data=(X_val, Y_val),
    # callbacks=[tensorboard_callback],
    verbose=1,
)

ckpt_save_path = ckpt_manager.save()
pprint(f"    - Saving checkpoint to:", ckpt_save_path)

Epoch 1/300
116/116 [==============================] - 5s 44ms/step - loss: 0.6832 - tp: 3227.0000 - fp: 22595.0000 - tn: 30388.0000 - fn: 2876.0000 - accuracy: 0.5689 - precision: 0.1250 - recall: 0.5288 - mcc: 0.0628 - auc: 0.5739 - val_loss: 0.6255 - val_tp: 424.0000 - val_fp: 1392.0000 - val_tn: 5057.0000 - val_fn: 422.0000 - val_accuracy: 0.7513 - val_precision: 0.2335 - val_recall: 0.5012 - val_mcc: 0.2113 - val_auc: 0.7337
Epoch 2/300
116/116 [==============================] - 4s 38ms/step - loss: 0.6469 - tp: 3601.0000 - fp: 18280.0000 - tn: 34703.0000 - fn: 2502.0000 - accuracy: 0.6483 - precision: 0.1646 - recall: 0.5900 - mcc: 0.1544 - auc: 0.6680 - val_loss: 0.5887 - val_tp: 469.0000 - val_fp: 1465.0000 - val_tn: 4984.0000 - val_fn: 377.0000 - val_accuracy: 0.7475 - val_precision: 0.2425 - val_recall: 0.5544 - val_mcc: 0.2374 - val_auc: 0.7494
Epoch 3/300
116/116 [==============================] - 5s 40ms/step - loss: 0.6250 - tp: 3780.0000 - fp: 16684.0000 - tn: 36299.0000

Epoch 20/300
116/116 [==============================] - 5s 39ms/step - loss: 0.5490 - tp: 4444.0000 - fp: 15139.0000 - tn: 37844.0000 - fn: 1659.0000 - accuracy: 0.7157 - precision: 0.2269 - recall: 0.7282 - mcc: 0.2860 - auc: 0.7911 - val_loss: 0.5372 - val_tp: 626.0000 - val_fp: 1804.0000 - val_tn: 4645.0000 - val_fn: 220.0000 - val_accuracy: 0.7225 - val_precision: 0.2576 - val_recall: 0.7400 - val_mcc: 0.3126 - val_auc: 0.8016
Epoch 21/300
116/116 [==============================] - 5s 39ms/step - loss: 0.5483 - tp: 4485.0000 - fp: 15380.0000 - tn: 37603.0000 - fn: 1618.0000 - accuracy: 0.7123 - precision: 0.2258 - recall: 0.7349 - mcc: 0.2864 - auc: 0.7915 - val_loss: 0.5221 - val_tp: 613.0000 - val_fp: 1727.0000 - val_tn: 4722.0000 - val_fn: 233.0000 - val_accuracy: 0.7313 - val_precision: 0.2620 - val_recall: 0.7246 - val_mcc: 0.3133 - val_auc: 0.8030
Epoch 22/300
116/116 [==============================] - 5s 43ms/step - loss: 0.5451 - tp: 4431.0000 - fp: 14983.0000 - tn: 38000.0

Epoch 39/300
116/116 [==============================] - 5s 41ms/step - loss: 0.5049 - tp: 4794.0000 - fp: 15002.0000 - tn: 37981.0000 - fn: 1309.0000 - accuracy: 0.7239 - precision: 0.2422 - recall: 0.7855 - mcc: 0.3239 - auc: 0.8270 - val_loss: 0.4799 - val_tp: 657.0000 - val_fp: 1654.0000 - val_tn: 4795.0000 - val_fn: 189.0000 - val_accuracy: 0.7474 - val_precision: 0.2843 - val_recall: 0.7766 - val_mcc: 0.3580 - val_auc: 0.8356
Epoch 40/300
116/116 [==============================] - 5s 43ms/step - loss: 0.5029 - tp: 4811.0000 - fp: 14680.0000 - tn: 38303.0000 - fn: 1292.0000 - accuracy: 0.7297 - precision: 0.2468 - recall: 0.7883 - mcc: 0.3309 - auc: 0.8287 - val_loss: 0.5257 - val_tp: 703.0000 - val_fp: 1889.0000 - val_tn: 4560.0000 - val_fn: 143.0000 - val_accuracy: 0.7215 - val_precision: 0.2712 - val_recall: 0.8310 - val_mcc: 0.3600 - val_auc: 0.8378
Epoch 41/300
116/116 [==============================] - 5s 42ms/step - loss: 0.5009 - tp: 4794.0000 - fp: 14672.0000 - tn: 38311.0

Epoch 58/300
116/116 [==============================] - 5s 46ms/step - loss: 0.4686 - tp: 4951.0000 - fp: 13778.0000 - tn: 39205.0000 - fn: 1152.0000 - accuracy: 0.7473 - precision: 0.2643 - recall: 0.8112 - mcc: 0.3605 - auc: 0.8545 - val_loss: 0.4801 - val_tp: 701.0000 - val_fp: 1706.0000 - val_tn: 4743.0000 - val_fn: 145.0000 - val_accuracy: 0.7463 - val_precision: 0.2912 - val_recall: 0.8286 - val_mcc: 0.3841 - val_auc: 0.8618
Epoch 59/300
116/116 [==============================] - 6s 49ms/step - loss: 0.4711 - tp: 4905.0000 - fp: 13839.0000 - tn: 39144.0000 - fn: 1198.0000 - accuracy: 0.7455 - precision: 0.2617 - recall: 0.8037 - mcc: 0.3548 - auc: 0.8528 - val_loss: 0.4850 - val_tp: 706.0000 - val_fp: 1725.0000 - val_tn: 4724.0000 - val_fn: 140.0000 - val_accuracy: 0.7443 - val_precision: 0.2904 - val_recall: 0.8345 - val_mcc: 0.3852 - val_auc: 0.8628
Epoch 60/300
116/116 [==============================] - 6s 49ms/step - loss: 0.4669 - tp: 4929.0000 - fp: 13506.0000 - tn: 39477.0

116/116 [==============================] - 5s 40ms/step - loss: 0.4476 - tp: 5000.0000 - fp: 13056.0000 - tn: 39927.0000 - fn: 1103.0000 - accuracy: 0.7604 - precision: 0.2769 - recall: 0.8193 - mcc: 0.3785 - auc: 0.8691 - val_loss: 0.4612 - val_tp: 708.0000 - val_fp: 1637.0000 - val_tn: 4812.0000 - val_fn: 138.0000 - val_accuracy: 0.7567 - val_precision: 0.3019 - val_recall: 0.8369 - val_mcc: 0.3997 - val_auc: 0.8752
Epoch 77/300
116/116 [==============================] - 5s 41ms/step - loss: 0.4466 - tp: 5005.0000 - fp: 12984.0000 - tn: 39999.0000 - fn: 1098.0000 - accuracy: 0.7617 - precision: 0.2782 - recall: 0.8201 - mcc: 0.3803 - auc: 0.8697 - val_loss: 0.4419 - val_tp: 703.0000 - val_fp: 1535.0000 - val_tn: 4914.0000 - val_fn: 143.0000 - val_accuracy: 0.7700 - val_precision: 0.3141 - val_recall: 0.8310 - val_mcc: 0.4117 - val_auc: 0.8765
Epoch 78/300
116/116 [==============================] - 5s 40ms/step - loss: 0.4454 - tp: 5016.0000 - fp: 13010.0000 - tn: 39973.0000 - fn: 108

...

Epoch 151/300
116/116 [==============================] - 5s 41ms/step - loss: 0.3840 - tp: 5202.0000 - fp: 10766.0000 - tn: 42217.0000 - fn: 901.0000 - accuracy: 0.8025 - precision: 0.3258 - recall: 0.8524 - mcc: 0.4449 - auc: 0.9058 - val_loss: 0.3957 - val_tp: 725.0000 - val_fp: 1282.0000 - val_tn: 5167.0000 - val_fn: 121.0000 - val_accuracy: 0.8077 - val_precision: 0.3612 - val_recall: 0.8570 - val_mcc: 0.4719 - val_auc: 0.9060
Epoch 152/300
116/116 [==============================] - 5s 41ms/step - loss: 0.3830 - tp: 5210.0000 - fp: 10578.0000 - tn: 42405.0000 - fn: 893.0000 - accuracy: 0.8059 - precision: 0.3300 - recall: 0.8537 - mcc: 0.4498 - auc: 0.9064 - val_loss: 0.4243 - val_tp: 751.0000 - val_fp: 1443.0000 - val_tn: 5006.0000 - val_fn: 95.0000 - val_accuracy: 0.7892 - val_precision: 0.3423 - val_recall: 0.8877 - val_mcc: 0.4636 - val_auc: 0.9066
Epoch 153/300
116/116 [==============================] - 5s 41ms/step - loss: 0.3815 - tp: 5223.0000 - fp: 10610.0000 - tn: 42373.0

Epoch 170/300
116/116 [==============================] - 5s 41ms/step - loss: 0.3705 - tp: 5242.0000 - fp: 10139.0000 - tn: 42844.0000 - fn: 861.0000 - accuracy: 0.8138 - precision: 0.3408 - recall: 0.8589 - mcc: 0.4630 - auc: 0.9130 - val_loss: 0.3946 - val_tp: 738.0000 - val_fp: 1273.0000 - val_tn: 5176.0000 - val_fn: 108.0000 - val_accuracy: 0.8107 - val_precision: 0.3670 - val_recall: 0.8723 - val_mcc: 0.4836 - val_auc: 0.9111
Epoch 171/300
116/116 [==============================] - 5s 41ms/step - loss: 0.3697 - tp: 5262.0000 - fp: 10200.0000 - tn: 42783.0000 - fn: 841.0000 - accuracy: 0.8131 - precision: 0.3403 - recall: 0.8622 - mcc: 0.4637 - auc: 0.9133 - val_loss: 0.3728 - val_tp: 727.0000 - val_fp: 1137.0000 - val_tn: 5312.0000 - val_fn: 119.0000 - val_accuracy: 0.8278 - val_precision: 0.3900 - val_recall: 0.8593 - val_mcc: 0.5014 - val_auc: 0.9118
Epoch 172/300
116/116 [==============================] - 5s 44ms/step - loss: 0.3708 - tp: 5230.0000 - fp: 10188.0000 - tn: 42795.

Epoch 189/300
116/116 [==============================] - 5s 41ms/step - loss: 0.3577 - tp: 5260.0000 - fp: 9703.0000 - tn: 43280.0000 - fn: 843.0000 - accuracy: 0.8215 - precision: 0.3515 - recall: 0.8619 - mcc: 0.4750 - auc: 0.9189 - val_loss: 0.3997 - val_tp: 749.0000 - val_fp: 1274.0000 - val_tn: 5175.0000 - val_fn: 97.0000 - val_accuracy: 0.8121 - val_precision: 0.3702 - val_recall: 0.8853 - val_mcc: 0.4919 - val_auc: 0.9162
Epoch 190/300
116/116 [==============================] - 5s 41ms/step - loss: 0.3589 - tp: 5261.0000 - fp: 9636.0000 - tn: 43347.0000 - fn: 842.0000 - accuracy: 0.8227 - precision: 0.3532 - recall: 0.8620 - mcc: 0.4767 - auc: 0.9186 - val_loss: 0.3517 - val_tp: 723.0000 - val_fp: 1051.0000 - val_tn: 5398.0000 - val_fn: 123.0000 - val_accuracy: 0.8391 - val_precision: 0.4076 - val_recall: 0.8546 - val_mcc: 0.5162 - val_auc: 0.9161
Epoch 191/300
116/116 [==============================] - 5s 41ms/step - loss: 0.3587 - tp: 5263.0000 - fp: 9610.0000 - tn: 43373.0000

Epoch 208/300
116/116 [==============================] - 5s 41ms/step - loss: 0.3468 - tp: 5307.0000 - fp: 9104.0000 - tn: 43879.0000 - fn: 796.0000 - accuracy: 0.8324 - precision: 0.3683 - recall: 0.8696 - mcc: 0.4945 - auc: 0.9241 - val_loss: 0.3964 - val_tp: 754.0000 - val_fp: 1239.0000 - val_tn: 5210.0000 - val_fn: 92.0000 - val_accuracy: 0.8175 - val_precision: 0.3783 - val_recall: 0.8913 - val_mcc: 0.5024 - val_auc: 0.9202
Epoch 209/300
116/116 [==============================] - 5s 41ms/step - loss: 0.3462 - tp: 5308.0000 - fp: 9374.0000 - tn: 43609.0000 - fn: 795.0000 - accuracy: 0.8279 - precision: 0.3615 - recall: 0.8697 - mcc: 0.4879 - auc: 0.9244 - val_loss: 0.3453 - val_tp: 726.0000 - val_fp: 1014.0000 - val_tn: 5435.0000 - val_fn: 120.0000 - val_accuracy: 0.8446 - val_precision: 0.4172 - val_recall: 0.8582 - val_mcc: 0.5266 - val_auc: 0.9204
Epoch 210/300
116/116 [==============================] - 5s 41ms/step - loss: 0.3465 - tp: 5264.0000 - fp: 9143.0000 - tn: 43840.0000

Epoch 227/300
116/116 [==============================] - 5s 45ms/step - loss: 0.3359 - tp: 5323.0000 - fp: 8953.0000 - tn: 44030.0000 - fn: 780.0000 - accuracy: 0.8353 - precision: 0.3729 - recall: 0.8722 - mcc: 0.5000 - auc: 0.9286 - val_loss: 0.3625 - val_tp: 735.0000 - val_fp: 1060.0000 - val_tn: 5389.0000 - val_fn: 111.0000 - val_accuracy: 0.8395 - val_precision: 0.4095 - val_recall: 0.8688 - val_mcc: 0.5237 - val_auc: 0.9230
Epoch 228/300
116/116 [==============================] - 5s 41ms/step - loss: 0.3373 - tp: 5291.0000 - fp: 8952.0000 - tn: 44031.0000 - fn: 812.0000 - accuracy: 0.8347 - precision: 0.3715 - recall: 0.8670 - mcc: 0.4966 - auc: 0.9281 - val_loss: 0.3769 - val_tp: 750.0000 - val_fp: 1145.0000 - val_tn: 5304.0000 - val_fn: 96.0000 - val_accuracy: 0.8299 - val_precision: 0.3958 - val_recall: 0.8865 - val_mcc: 0.5177 - val_auc: 0.9230
Epoch 229/300
116/116 [==============================] - 5s 42ms/step - loss: 0.3374 - tp: 5317.0000 - fp: 9050.0000 - tn: 43933.0000

In [ ]:
%%capture cap --no-stderr
"""
    Results
"""

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    pprint("Restoring checkpoint from:", ckpt_manager.latest_checkpoint)

print_variables(model.trainable_variables)
plot_metrics(training, Y_train, Y_val, save_dir=model_dir)

results_train = model.evaluate(X_train, Y_train, batch_size=n_batch, verbose=0)
results_val = model.evaluate(X_val, Y_val, batch_size=n_batch, verbose=0)
results_test = model.evaluate(X_test, Y_test, batch_size=n_batch, verbose=0)
results_test_all = model.evaluate(X_test_all, Y_test_all, batch_size=n_batch, verbose=0)

Y_train_pred = model.predict(X_train, batch_size=n_batch)
Y_val_pred = model.predict(X_val, batch_size=n_batch)
Y_test_pred = model.predict(X_test, batch_size=n_batch)
Y_test_all_pred = model.predict(X_test_all, batch_size=n_batch)

describe_results(model.metrics_names, results_train, Y_train, name="Train")
describe_results(model.metrics_names, results_val, Y_val, name="Validation")
describe_results(model.metrics_names, results_test, Y_test, name="Test")
describe_results(model.metrics_names, results_test_all, Y_test_all, name="Test-all")

plot_cm(Y_train, Y_train_pred, "Training", save_dir=model_dir)
plot_cm(Y_val, Y_val_pred, "Validation", save_dir=model_dir)
plot_cm(Y_test, Y_test_pred, "Test", save_dir=model_dir)

plot_roc(
    [
        ("Training", Y_train, Y_train_pred),
        ("Validation", Y_val, Y_val_pred),
        ("Test", Y_test, Y_test_pred),
        ("Test-all", Y_test_all, Y_test_all_pred),
    ],
    save_dir=model_dir,
)

In [ ]:
cap.show()

with open(os.path.join(model_dir, "log.txt"), "a") as f:
    f.write(cap.stdout)

In [ ]:
import matplotlib.pyplot as plt

from bclassification.utils_fcn import create_dataset

def plot_preds(t, y, y_pred, rhos, threshold, chronic_idx=None):
    fig, ax = plt.subplots(figsize=(Const.FIG_SIZE[0] * 2, Const.FIG_SIZE[1]))

    indices_pos = np.equal(y, 1)
    # ax.plot(t, y, label=r"$y$", color="tab:blue")
    ax.bar(t[indices_pos], y[indices_pos], label=r"$y$", color="tab:blue", lw=1.5)

    # ax.plot(t, y_pred, label=r"$P(y = 1)$", color="tab:green")
    indices_pos = np.greater(y_pred, threshold)
    pos = np.ma.masked_where(np.greater_equal(y_pred, threshold), y_pred)
    neg = np.ma.masked_where(np.less_equal(y_pred, threshold), y_pred)
    ax.plot(t, pos, "tab:green", t, neg, "tab:red")

    ax.plot(t, np.ones_like(t) * threshold, label=r"$y$", color="tab:red")
    # ax.plot(t, rhos, label=r"$y$")

    # ax.legend()
    ax.set_xlabel("Time step t")
    ax.set_ylabel(r"$P(y = 1)$")

    if ax.get_xlim()[-1] > 2000:
        ax.set_xlim(right=2000, left=-10)
    else:
        ax.set_xlim(left=-10)

    fig.tight_layout()
    if model_dir and not isinstance(chronic_idx, type(None)):
        fig.savefig(os.path.join(model_dir, "test-y-step-{:04}".format(chronic_idx)))
        
    if not isinstance(rhos, type(None)):
        ax2 = ax.twinx()
        ax2.set_ylabel(r"$\rho^\mathrm{max}$")
        ax2.plot(t, rhos, label=r"$y$", lw=Const.LW, color="tab:orange")
        ax2.set_ylim(*ax.get_ylim())

    fig.tight_layout()
    if model_dir and not isinstance(chronic_idx, type(None)):
        fig.savefig(os.path.join(model_dir, "test-y-step-{:04}-rhos".format(chronic_idx)))


sample_experience_dir = make_dir(os.path.join(Const.EXPERIENCE_DIR, "data-aug-sample"))
case, collector = load_experience(case_name, agent_name, sample_experience_dir, env_dc=env_dc)

_, _, _, X_all_sample, Y_all_sample = create_dataset(
    case,
    collector,
    input_mode=input_mode,
    label_mode=label_mode,
    n_window_history=n_window_history,
    n_window_targets=n_window_targets,
    downsampling_rate=downsampling_rate,
    n_window_forecasts=n_window_forecasts,
    use_actions=use_actions,
    feature_scaling=feature_scaling,
)

In [ ]:
start_idx = 0
for chronic_idx, chronic_len in zip(collector.chronic_ids, collector.chronic_lengths):
    X_chronic = X_all_sample[start_idx:(start_idx + chronic_len), :]
    Y_chronic = Y_all_sample[start_idx:(start_idx + chronic_len)]
    Y_chronic_pred = model.predict(X_chronic, batch_size=n_batch).flatten()
    
    t = np.arange(chronic_len)
    rhos = [np.max(obs.rho) for obs in collector.data[chronic_idx]["obses"][:-1]]
        
    plot_preds(t, Y_chronic, Y_chronic_pred, rhos, threshold, chronic_idx)
    
    start_idx = start_idx + chronic_len

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import TSNE

def plot_scatter(ax, data, color_label, marker_label=None, x_label=None, y_label=None, ax_title=None):
    colors = Const.COLORS
    
    color_values = sorted(data[color_label].unique())
    
    for i, color_val in enumerate(color_values):
        color_id = i % len(colors)
        color = colors[color_id]
        
        if marker_label:
            marker_values = sorted(data[marker_label].unique())
            for j, marker_val in enumerate(marker_values):
                d = data[np.logical_and(data[color_label] == color_val, data[marker_label] == marker_val)]

                if d.shape[0] > 0:
                    if marker_val == "1":
                        ax.scatter(d[x_label], d[y_label], label=str(color_val), s=50, marker="+", facecolors='none', c=color)
                    elif marker_val == "0":
                        ax.scatter(d[x_label], d[y_label], label=str(color_val), s=30, marker="o", facecolors='none', edgecolors=color, alpha=0.5)
                    else:
                        ax.scatter(d[x_label], d[y_label], label=str(color_val), s=30, marker="+", c=color)                
        else:
            d = data[data[color_label] == color_val]
            ax.scatter(d[x_label], d[y_label], label=str(color_val), s=30, marker="+", c=color)
            
    if len(color_values) < 4 and not marker_label:
        ax.legend(color_values)
        
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    
    if ax_title:
        ax.set_title(ax_title)

"""
    TSNE

    Following: https://towardsdatascience.com/visualising-high-dimensional-datasets-using-pca-and-t-sne-in-python-8ef87e7915b
"""

file_name = f"tsne-d{dr_str}-h{h_str}-f{f_str}{ft_str}-{input_mode}.npz"
file_path = os.path.join(experience_dir, file_name)


if file_name in os.listdir(experience_dir):
    pprint("Loading:", file_path)
    tsne_results = np.load(file_path)["tsne_results"]
else:
    tsne = TSNE(n_components=2, verbose=1, perplexity=30, n_iter=300)
    red = PCA(n_components=100, random_state=random_seed)

    X_red = red.fit_transform(X)
    pprint("    - Explained variance:", red.explained_variance_ratio_.sum())
    tsne_results = tsne.fit_transform(X_red)
    np.savez_compressed(
        file_path,
        tsne_results=tsne_results,
    )
    pprint("Saving:", file_path)

data = pd.DataFrame()
data["tsne-1"] = tsne_results[:, 0]
data["tsne-2"] = tsne_results[:, 1]
data["y"] = Y.astype(int).astype(str)

data_train, data_val = train_test_split(
    data, test_size=val_frac, random_state=random_seed
)

data_train, data_test = train_test_split(
    data_train, test_size=test_frac, random_state=random_seed
)

fig_name = "tsne-X-train"
fig, ax = plt.subplots(figsize=Const.FIG_SIZE)
plot_scatter(ax, data_train, color_label="y", x_label="tsne-1", y_label="tsne-2")
fig.savefig(os.path.join(model_dir, fig_name))

fig_name = "tsne-X-val"
fig, ax = plt.subplots(figsize=Const.FIG_SIZE)
plot_scatter(ax, data_val, color_label="y", x_label="tsne-1", y_label="tsne-2")
fig.savefig(os.path.join(model_dir, fig_name))

fig_name = "tsne-X-test"
fig, ax = plt.subplots(figsize=Const.FIG_SIZE)
plot_scatter(ax, data_test, color_label="y", x_label="tsne-1", y_label="tsne-2")
fig.savefig(os.path.join(model_dir, fig_name))


Y_pred = np.greater(model.predict(X, batch_size=n_batch), threshold).astype(int).flatten()
data["y"][np.logical_and(np.not_equal(Y, Y_pred), np.equal(Y, 1))] = "2"

data_train, data_val = train_test_split(
    data, test_size=val_frac, random_state=random_seed
)

data_train, data_test = train_test_split(
    data_train, test_size=test_frac, random_state=random_seed
)

fig_name = "tsne-X-train"
fig, ax = plt.subplots(figsize=Const.FIG_SIZE)
plot_scatter(ax, data_train, color_label="y", x_label="tsne-1", y_label="tsne-2")
fig.savefig(os.path.join(model_dir, fig_name))

fig_name = "tsne-X-labels-val"
fig, ax = plt.subplots(figsize=Const.FIG_SIZE)
plot_scatter(ax, data_val, color_label="y", x_label="tsne-1", y_label="tsne-2")
fig.savefig(os.path.join(model_dir, fig_name))

fig_name = "tsne-X-labels-test"
fig, ax = plt.subplots(figsize=Const.FIG_SIZE)
plot_scatter(ax, data_test, color_label="y", x_label="tsne-1", y_label="tsne-2")
fig.savefig(os.path.join(model_dir, fig_name))